# Equação de Calor
----

## Bibliotecas

Utilizaremos o DeepXDE junto com PyTorch para resolver estas equações.

In [13]:
import torch
import deepxde as dde
import numpy as np

In [ ]:
# Para reprodução dos experimentos
dde.config.set_random_seed(1000)

## Modelo equação de calor

$$
\frac{\partial u}{\partial t}(x,t) - \alpha \frac{\partial^2 u}{\partial x^2}(x,t) = f(x,t), \quad x \in (0,\ell), \ t> 0
$$

In [3]:
a = 0.4

In [4]:
def pde(x, y):
    dy_dt = dde.grad.jacobian(y, x, i=0, j=1)
    dy_dxx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_dt - a * dy_dxx

## Limites

$$
x \in (0,\ell), \ t> 0
$$

In [5]:
L = 1
n = 1

In [6]:
geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

## Condições de contorno e iniciais

In [7]:
bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)

ic = dde.icbc.IC(
    geomtime,
    lambda x: np.sin(n * np.pi * x[:, 0:1] / L),
    lambda _, on_initial: on_initial,
)

## Solução Exata

$$
u(x,t) = e^{\frac{-\alpha \pi^2 t}{\ell}} \sin \Big( \frac{\pi x}{\ell} \Big)
$$

In [15]:
def exact_solution(x, t):
    return np.exp(-(n**2 * np.pi**2 * a * t) / (L**2)) * np.sin(n * np.pi * x / L)

## Dados de treinamento

In [10]:
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=2540,
    num_boundary=80,
    num_initial=160,
    num_test=2540,
)

## Otimizando a rede

In [11]:
n_hidden_layers = [2, 3, 4]
n_nodes_per_layer = [20, 30, 40]

In [14]:
for n_hidden_l in n_hidden_layers:
    for n_nodes in n_nodes_per_layer:
        
        net = dde.nn.FNN([2] + [n_nodes] * n_hidden_l + [1], "tanh", "Glorot normal")
        model = dde.Model(data, net)
        
        model.compile("adam", lr=1e-3)
        losshistory, train_state = model.train(iterations=20000)
        
        dde.saveplot(
            losshistory, 
            train_state, 
            output_dir="../treinamentos",
            loss_fname=f"heat_{n_nodes}x{n_hidden_l}_losses.dat",
            train_fname=f"heat_{n_nodes}x{n_hidden_l}_train.dat",
            test_fname=f"heat_{n_nodes}x{n_hidden_l}_test.dat",
            issave=True, 
            isplot=False)
        

Compiling model...
'compile' took 0.000186 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [4.67e-02, 1.85e-02, 4.84e-01]    [4.82e-02, 1.85e-02, 4.84e-01]    []  
1000      [5.45e-04, 2.56e-04, 1.24e-04]    [2.88e-04, 2.56e-04, 1.24e-04]    []  
2000      [1.71e-04, 4.23e-05, 2.57e-05]    [9.77e-05, 4.23e-05, 2.57e-05]    []  
3000      [6.80e-05, 8.67e-06, 5.82e-06]    [4.96e-05, 8.67e-06, 5.82e-06]    []  
4000      [3.85e-05, 3.36e-06, 1.47e-06]    [2.51e-05, 3.36e-06, 1.47e-06]    []  
5000      [9.10e-05, 2.67e-05, 7.30e-05]    [8.51e-05, 2.67e-05, 7.30e-05]    []  
6000      [2.13e-05, 2.45e-06, 5.63e-07]    [1.39e-05, 2.45e-06, 5.63e-07]    []  
7000      [2.31e-05, 4.10e-06, 1.05e-05]    [1.62e-05, 4.10e-06, 1.05e-05]    []  
8000      [1.53e-05, 1.69e-06, 4.42e-07]    [9.98e-06, 1.69e-06, 4.42e-07]    []  
9000      [1.45e-05, 2.30e-06, 3.77e-06]    [9.76e-06, 2.30e-06, 3.77e-06]    []  
10000     [1.19